In [ ]:
import re
import pandas as pd
from pathlib import Path
from os import system, name

In [ ]:
class Grammar:
    def __init__(self, rules):
        rules = tuple(rules)
        self.rules = tuple(self._parse(rule) for rule in rules)

    def _parse(self, rule):
        return tuple(rule.replace(' ', '').split('->'))

    @property
    def nonterminals(self):
        return set([nt for nt, _ in self.rules])

    @property
    def terminals(self):
        a = [
            expression
            for _, expression in self.rules
        ]
        
        nt = sorted([nt for nt, _ in self.rules], key=len, reverse=True)
        a = [i for j in [tr3(i, nt) for i in a] for i in j]

        for i in nt:
            for j in range(0, len(a)):
                if re.search(i, a[j]):
                    a[j] = a[j].replace(i, '')
                a[j] = " ".join(a[j].split())

        tmp = []
        for i in a:
            if i != '' and i not in tmp:
                tmp.append(i)
        a = tmp

        tmp = []
        for i in a:
            if len(i) > 1 and not i.isalpha():
                tmp2 = []
                tmp2.append(list(i))
                tmp2 = [i for j in tmp2 for i in j]
                for j in tmp2:
                    tmp.append(j)
            else:
                tmp.append(i)
        a = tmp

        tmp = []
        
        a = [i for j in [list(i) for i in a] for i in j] # Lugar Onde Comentar ou Descomentar De Acordo com a gramatica
        return set(a)

In [ ]:
def union(first, begins):
    n = len(first)
    first |= begins
    return len(first) != n

In [ ]:
def clear():
    if name == 'nt':
        system('cls')
    else:
        system('clear')

In [ ]:
def transform2(string, lista):  # conversão e filtragem
    listanova = []
    texto = ""
    tam = len(string)
    x = 0
    while x < tam:
        texto += string[x]
        if(x < tam-1 and texto+string[x+1] in lista):
            listanova.append(texto+string[x+1])
            texto = ""
            x += 1
        elif texto in lista:
            listanova.append(texto)
            texto = ""
        x += 1
    return listanova

In [ ]:
def tr3(string, lista):  # conversão e filtragem
    stringnova = ""
    tam = len(string)
    x = 0
    while x < tam:
        if(x < tam-1 and str(string[x]) + str(string[x+1]) in lista):
            stringnova += str(" ")+str(string[x])+str(string[x+1])+str(" ")
            x += 1
        elif(str(string[x]) in lista):
            stringnova += " "+str(string[x])+" "
        else:
            stringnova += str(string[x])
        x += 1
    return list(filter(lambda x: x != "", stringnova.split(" ")))

In [ ]:
def verificar_palavra(string, tabela, grammar):
    lista = sorted(
        (list(grammar.terminals)+list(grammar.nonterminals)), key=len, reverse=True)
    lista1 = sorted(list(lista)+list('$'), key=len, reverse=True)
    rule = [(i, transform2(j, lista)) for i, j in grammar.rules]
    nao_terminais = list(grammar.nonterminals)
    terminais = list(grammar.terminals)
    string_pilha = transform2(string, lista)

    if '^' in rule[0][0]:
        rule.pop(0)
    rule = tuple(rule)
    nt_pilha = [rule[0][0]]

    nt_pilha.append('$')
    string_pilha.append('$')
    top = nt_pilha[0]

    verifica = set(tr3(string, lista))
    verifica2 = set(transform2(string, lista))
    if verifica != verifica2:
        return "ERRO String Inválida!!!!"

    resultado = []

    while True:
        if not (top == string_pilha[0]) and nt_pilha[0] in terminais:
            resultado.append([str(nt_pilha) ,str(string_pilha),"String Recusada"])
            return "String Recusada", resultado
        if top == '$' and string_pilha[0] == '$':
            resultado.append([str(nt_pilha) ,str(string_pilha),"Sentença OK"])
            return "String Aceita", resultado
        if top == string_pilha[0]:
            resultado.append([str(nt_pilha), str(string_pilha),"Desempilha {}".format(nt_pilha[0])])
            nt_pilha.pop(0)
            string_pilha.pop(0)
            top = nt_pilha[0]
        if top in nao_terminais:
            ant_top = top
            consulta = tabela[(top, string_pilha[0])]
            if consulta == 'ERRO':
                resultado.append([str(nt_pilha), str(string_pilha) ,"ERRO - String Recusada Pela Tabela"]) 
                return "ERRO - String Recusada Pela Tabela", resultado
            else:
                resultado.append([str(nt_pilha), str(string_pilha),"{} -> {}".format(ant_top, consulta)])
                nt_pilha[0] = consulta
                if len(consulta) > 1:
                    nt_pilha = [i for j in [transform2(
                        i, lista1) for i in nt_pilha] for i in j]
                if nt_pilha[0] == 'ε':
                    nt_pilha.pop(0)
                top = nt_pilha[0]

In [ ]:
def LL1(first, follow, grammar):
    lista = sorted(
        (list(grammar.terminals)+list(grammar.nonterminals)), key=len, reverse=True)
    rule = [(i, transform2(j, lista)) for i, j in grammar.rules]
    terminais = grammar.terminals - {'ε'}
    if not ('^' in rule[0][0]):
        valor = rule[0][0]+'$'
        rule.insert(0, ('^', valor))
        lista = list(lista) + list('$') + list('^')
        terminais |= {'$'}
    if '^' in rule[0][0]:
        rule.pop(0)
    rule = tuple(rule)
    terminais = sorted(terminais, key=len, reverse=True)

    table = {}
    for nt, expression in rule:
        for element in list(terminais):
            table[nt, element] = 'ERRO'
    for nt, expression in rule:
        first_set = first[nt]
        for element in (first_set-{'ε'}):
            for symbol in expression:
                if element in first[symbol]:
                    table[nt, element] = "".join(expression)
        if 'ε' in first_set:
            for element in follow[nt]:
                table[nt, element] = "".join(expression)
        if 'ε' in first[nt] and '$' in follow[nt]:
            table[nt, '$'] = "".join(expression)
    return table

In [ ]:
def FirstAndFollow(terminais, nao_terminais, regras):
    lista = sorted((list(terminais)+list(nao_terminais)),
                   key=len, reverse=True)
    first = {i: set() for i in nao_terminais}
    first.update((i, {i}) for i in terminais)
    follow = {i: set() for i in nao_terminais}

    rule = tuple([(i, transform2(j, lista)) for i, j in regras])

    if not ('^' in rule[0][0]):
        rule = [list(i) for i in rule]
        valor = [rule[0][0], '$']
        rule.insert(0, ['^', valor])
        lista = sorted((list(lista) + list('$') + list('^')),
                       key=len, reverse=True)
        terminais |= {'$'}
        nao_terminais |= {'^'}
        rule = tuple(tuple(i) for i in rule)
        first = {i: set() for i in nao_terminais}
        first.update((i, {i}) for i in terminais)
        follow = {i: set() for i in nao_terminais}

    epsilon = {'ε'}

    while True:
        updated = False
        for nt, expression in rule:
            for symbol in expression:
                updated |= union(first[nt], first[symbol])
                if symbol not in epsilon:
                    break
                else:
                    updated |= union(epsilon, {nt})
            aux = follow[nt]
            for symbol in reversed(expression):
                if symbol in follow:
                    updated |= union(follow[symbol], aux)
                if symbol in epsilon:
                    aux = aux.union(first[symbol])
                else:
                    aux = first[symbol]

        if not updated:
            for chave, valor in follow.items():
                if 'ε' in follow[chave]:
                    follow[chave] = follow[chave] - {'ε'}
            cond1 = False
            cond2 = False
            for i in epsilon:
                if '^' in i:
                    cond1 = True
                if 'ε' in i:
                    cond2 = True

            for i in nao_terminais:
                if '^' in i:
                    first.pop(i)
                    follow.pop(i)
                if cond1 and '^' in i:
                    epsilon.remove(i)

            for i in terminais:
                # first.pop(i)
                first['$'] = '$'
                if cond2 and 'ε' in i:
                    epsilon.remove(i)
            for i in {'$'}:
                first.pop(i)

            return first, follow, epsilon


In [ ]:
def retornar_tabela(table):
    new_table = {}
    for pair in table:
        new_table[pair[1]] = {}
    for pair in table:
        new_table[pair[1]][pair[0]] = table[pair]
    df = pd.DataFrame(new_table).fillna('ERRO')
    return df

In [ ]:
def visualizar_tabela(table):
    print("\nTabela Sintática Preditiva \n")
    print(table)
    print("\n")

In [ ]:

def visual_tab(table):
    table2 = {}
    for key, val in table.items():
        table2[key] = 'ERRO'
        if not ('ERRO' in val):
            table2[key] = ["{} -> {}".format(key[0], val)]
    return table2

In [ ]:

def visualizar_first(first):
    print("First\n")
    for key, value in first.items():
        print("First({}) = {}".format(key, value))
    print("\n--------------------------------------------------------------------------------")

In [ ]:

def visualizar_follow(follow):
    print("Follow\n")
    for key, value in follow.items():
        print("Follow({}) = {}".format(key, value))
    print("\n--------------------------------------------------------------------------------")

In [ ]:
def visualizar_grammar(grammar):
    clear()
    print("\n\nGramatica\n")
    for i in grammar:
        print(i)
    print("\n--------------------------------------------------------------------------------")

In [ ]:

def limpar_terminais_first(terminais, first):
    for i in terminais - {'$'}:
        if i in first:
            first.pop(i)

In [ ]:
def tratarArq(lista):
    lista = " ".join(lista.split())
    lista = re.split(r' ', lista)
    return lista

In [ ]:

def lerArquivo(dir):
    arquivo = []
    b = []
    inicio = ""
    fim = ""

    with open(dir, "r") as gramatica:
        for line in gramatica:
            arquivo.append(line.replace('Îµ', 'ε').strip().split('\n'))
    arquivo = [i for j in arquivo for i in j]

    for i in range(0, len(arquivo)):
        temp = []
        temp = tratarArq(arquivo[i])
        arquivo[i] = temp

    for i in range(0, len(arquivo)):
        tam = len(arquivo[i])
        for j in range(0, tam):
            if "|" in arquivo[i][j]:
                inicio = arquivo[i][0]+" "+arquivo[i][1]
                fim = arquivo[i][-1]
                suma = inicio+" "+fim
                arquivo.append(tratarArq(suma))

    for i in arquivo:
        b.append(" ".join(i[:3]))
    return b

In [ ]:
def salvarArquivoFirst(first, epsilon):
    outpath = 'output'
    Path(outpath).mkdir(exist_ok=True)
    with open(outpath+'/First.txt', 'w+', encoding="utf-8") as file:
        a = "Epsilon na Gramatica = {}".format(epsilon)
        file.write(a + '\n')
        for i in first:
            a = "Primeiro({}) = {}".format(i, first[i])
            file.write(a + '\n')

In [ ]:
def salvarArquivoFollow(follow):
    outpath = 'output'
    Path(outpath).mkdir(exist_ok=True)
    with open(outpath+'/Follow.txt', 'w+', encoding="utf-8") as file:
        a = ''
        for i in follow:
            a = "Follow({}) = {}".format(i, follow[i])
            file.write(a + '\n')

In [ ]:

def gravarArquivo(first, follow, epsilon):
    outpath = 'output'
    Path(outpath).mkdir(exist_ok=True)
    with open(outpath+'/First.txt', 'w+', encoding="utf-8") as file:
        a = "Epsilon na Gramatica = {}".format(epsilon)
        file.write(a + '\n')
        for i in first:
            a = "Primeiro({}) = {}".format(i, first[i])
            file.write(a + '\n')
    print("\n")
    with open(outpath+'/Follow.txt', 'w+', encoding="utf-8") as file:
        a = ''
        for i in follow:
            a = "Follow({}) = {}".format(i, follow[i])
            file.write(a + '\n')

In [ ]:
def printArquivo(dir):
    with open(dir, "r", encoding="utf-8") as file:
        for line in file:
            print(line)

In [ ]:

def salvar_tabela(tabela):
    outpath = 'output'
    Path(outpath).mkdir(exist_ok=True)
    tabela.to_excel(outpath+'/Tabela.xls')

In [ ]:

def tr_terminais(terminais):
    return [i for j in [list(i) for i in terminais] for i in j]

In [ ]:
def salvar_verificador(resultado):
    outpath = 'output'
    Path(outpath).mkdir(exist_ok=True)
    colunas = "Pilha Entrada Acão".split()
    dados = pd.DataFrame(data=resultado, columns=colunas)
    dados.to_excel(outpath+"/Verificador.xls")

In [ ]:
def visualizar_saida(first, follow, epsilon, tabela, gramatica):
    print('--------------------------------------------------------------------------------')
    X = True
    while X:
        print("\n")
        op = input('1 - Visualizar First\n2 - Visualizar Follow\n3 - Visualizar First e Follow\n4 - Visualizar Tabela\n5 - Verificar Cadeia\n6 - Sair\n\nDigite a Opção: ')
        print('\n--------------------------------------------------------------------------------\n')
        if op == '1':
            clear()
            limpar_terminais_first(gramatica.terminals, first)
            visualizar_first(first)

            opc1 = input(
                "Deseja Salvar em Arquivo ?:\n\t(Se Sim Digite :'s', Ou Digite Qualquer Coisa e Dê Enter para Não Salvar): ")
            if opc1 == 's':
                salvarArquivoFirst(first, epsilon)
                print("Arquivo Salvo em /output/First.txt")
            elif opc1 != 's':
                clear()
                pass

            opc = input(
                "Deseja voltar ao menu principal:\n\t(Se Sim Digite :'s', Ou Digite Qualquer Coisa e Dê Enter para Encerrar): ")
            if opc == 's':
                clear()
                X = True
            else:
                clear()
                X = False

        if op == '2':
            clear()
            visualizar_follow(follow)

            opc1 = input(
                "Deseja Salvar em Arquivo ?:\n\t(Se Sim Digite :'s', Ou Digite Qualquer Coisa e Dê Enter para Não Salvar): ")
            if opc1 == 's':
                salvarArquivoFollow(follow)
                print("Arquivo Salvo em /output/Follow.txt")
            elif opc1 != 's':
                clear()
                pass

            opc = input(
                "Deseja voltar ao menu principal:\n\t(Se Sim Digite :'s', Ou Digite Qualquer Coisa e Dê Enter para Encerrar): ")
            if opc == 's':
                clear()
                X = True
            else:
                clear()
                X = False

        if op == '3':
            clear()
            limpar_terminais_first(gramatica.terminals, first)
            visualizar_first(first)
            print(
                '--------------------------------------------------------------------------------\n')
            visualizar_follow(follow)

            opc1 = input(
                "Deseja Salvar em Arquivo ?:\n\t(Se Sim Digite :'s', Ou Digite Qualquer Coisa e Dê Enter para Não Salvar): ")
            if opc1 == 's':
                gravarArquivo(first, follow, epsilon)
                print("Arquivo Salvo em /output/First.txt")
                print("Arquivo Salvo em /output/Follow.txt")
            elif opc1 != 's':
                clear()
                pass

            opc = input(
                "Deseja voltar ao menu principal:\n\t(Se Sim Digite :'s', Ou Digite Qualquer Coisa e Dê Enter para Encerrar): ")
            if opc == 's':
                clear()
                X = True
            else:
                clear()
                X = False

        if op == '4':
            clear()
            visualizar_tabela(retornar_tabela(visual_tab(tabela)))

            opc1 = input(
                "Deseja Salvar em Arquivo ?:\n\t(Se Sim Digite :'s', Ou Digite Qualquer Coisa e Dê Enter para Não Salvar): ")
            if opc1 == 's':
                salvar_tabela(retornar_tabela(visual_tab(tabela)))
                print("Arquivo Salvo em /output/Tabela.xls")
            elif opc1 != 's':
                clear()
                pass

            opc = input(
                "Deseja voltar ao menu principal:\n\t(Se Sim Digite :'s', Ou Digite Qualquer Coisa e Dê Enter para Encerrar): ")
            if opc == 's':
                clear()
                X = True
            else:
                clear()
                X = False

        if op == '5':
            clear()
            verificador = verificar_palavra(
                input("Escreva uma sentença: "), tabela, gramatica)

            if type(verificador) == tuple:
                string, pilha = verificador

                col = "Pilha Entrada Ação".split()
                dados = pd.DataFrame(data=pilha, columns=col)

                print("\n")
                print(dados.set_index(['Pilha', 'Entrada']))
                print("\n")

                opc1 = input(
                    "Deseja Salvar em Arquivo ?:\n\t(Se Sim Digite :'s', Ou Digite Qualquer Coisa e Dê Enter para Não Salvar): ")
                if opc1 == 's':
                    salvar_verificador(pilha)
                    print("Arquivo Salvo em /output/Verificador.xls")
                elif opc1 != 's':
                    clear()
                    pass

            opc = input(
                "Deseja voltar ao menu principal:\n\t(Se Sim Digite :'s', Ou Digite Qualquer Coisa e Dê Enter para Encerrar): ")
            if opc == 's':
                clear()
                X = True
            else:
                clear()
                X = False

        if op == '6':
            clear()
            X = False

In [ ]:
def comentarios_sobre_gramaticas(escolha):
    if escolha == 1:
        print("Sobre a Gramatica a.txt, é necessário (((comentar))) \na penuntima linha da Função 'terminals' \nLocalizada em grammar.py Dentro da Classe Grammar \npara um bom funcionamento do código")
    if escolha == 2:
        print("Sobre a Gramatica b.txt, é necessário (((descomentar))) \na penuntima linha da Função 'terminals' \nLocalizada em grammar.py Dentro da Classe Grammar \npara um bom funcionamento do código")
    if escolha == 3:
        print("Sobre a Gramatica c.txt, é necessário (((descomentar))) \na penuntima linha da Função 'terminals' \nLocalizada em grammar.py Dentro da Classe Grammar \npara um bom funcionamento do código")
    if escolha == 4:
        print("Sobre a Gramatica d.txt, é necessário (((descomentar))) \na penuntima linha da Função 'terminals' \nLocalizada em grammar.py Dentro da Classe Grammar \npara um bom funcionamento do código")
    if escolha == 5:
        print("Sobre a Gramatica e.txt, é necessário (((descomentar))) \na penuntima linha da Função 'terminals' \nLocalizada em grammar.py Dentro da Classe Grammar \npara um bom funcionamento do código")
    if escolha == 6:
        print("Sobre a Gramatica f.txt, é necessário (((descomentar))) \na penuntima linha da Função 'terminals' \nLocalizada em grammar.py Dentro da Classe Grammar \npara um bom funcionamento do código")
    if escolha == 7:
        print("Sobre a Gramatica g.txt, é necessário (((descomentar))) \na penuntima linha da Função 'terminals' \nLocalizada em grammar.py Dentro da Classe Grammar \npara um bom funcionamento do código")
    if escolha == 8:
        print("Sobre a Gramatica h.txt, é necessário (((descomentar))) \na penuntima linha da Função 'terminals' \nLocalizada em grammar.py Dentro da Classe Grammar \npara um bom funcionamento do código")
    if escolha == 9:
        print("Sobre a Gramatica i.txt, é necessário (((comentar))) \na penuntima linha da Função 'terminals' \nLocalizada em grammar.py Dentro da Classe Grammar \npara um bom funcionamento do código")

In [ ]:
def main():
    dir = "gramaticas"

    gramatica = [dir+'/a.txt', dir+'/b.txt', dir+'/c.txt',
                 dir+'/d.txt', dir+'/e.txt', dir+'/f.txt', dir+'/g.txt', dir+'/h.txt', dir+'/i.txt']

    clear()
    escolha = int(input('Selecione Entre 1 até 9:\n1 - a.txt\n2 - b.txt\n3 - c.txt\n4 - d.txt\n5 - e.txt\n6 - f.txt\n7 - g.txt\n8 - h.txt(Principal)\n9 - i.txt\n\nEscolha uma Gramatica: '))

    if not (escolha in [i for i in range(1, 10)]):
        print("Escolha Invalida")
        exit()

    clear()

    comentarios_sobre_gramaticas(escolha)
    confirmar = input(
        "Digite OK para Continuar ou qualquer coisa para finalizar: ")
    if confirmar != 'OK':
        clear()
        exit()
    elif confirmar == 'OK':
        clear()
        pass

    gr = lerArquivo(gramatica[escolha-1])

    visualizar_grammar(gr)

    gramatica = Grammar(gr)

    terminais, nao_terminais, regras = gramatica.terminals, gramatica.nonterminals, gramatica.rules

    first, follow, epsilon = FirstAndFollow(terminais, nao_terminais, regras)

    tabela = LL1(first, follow, gramatica)

    print("Epsilon nas Expresões \n\n{}".format(epsilon))

    visualizar_saida(first, follow, epsilon, tabela, gramatica)

In [ ]:
main()

Selecione Entre 1 até 9:
1 - a.txt
2 - b.txt
3 - c.txt
4 - d.txt
5 - e.txt
6 - f.txt
7 - g.txt
8 - h.txt(Principal)
9 - i.txt

Escolha uma Gramatica: 1
Sobre a Gramatica a.txt, é necessário (((comentar))) 
a penuntima linha da Função 'terminals' 
Localizada em grammar.py Dentro da Classe Grammar 
para um bom funcionamento do código
Digite OK para Continuar ou qualquer coisa para finalizar: OK


FileNotFoundError: ignored